In [1]:
do_balance = True
do_populyarnost = True
do_dmg = True

In [2]:
names = ["Ling",
"Shang the Monk",
"Kate",
"Marcus",
"Ironclad",
"Hong-Joo",
"Fireguard",
"Kibo",
"Helga",
"Sarge",
"Azuma",
"Midnight",
"Yukka and Shade",
"Jack Bulwark",
"Jet",
"Emperor",
"Lynx",
"Monkey King",
"Cobra"
        ]

names_dmg = {
    "Ling": "Ling", 
    "Monk": "Shang the Monk",
    "Liquidator": "Kate",
    "Marcus": "Marcus",
    "Ironclad": "Ironclad",
    "HongJoo": "Hong-Joo",
    "Fireguard": "Fireguard",
    "Kibo": "Kibo",
    "Helga": "Helga",
    "Sarge": "Sarge",
    "Feldsher": "Azuma",
    "Midnight": "Midnight",
    "Yukka": "Yukka and Shade",
    "Bulwark": "Jack Bulwark",
    "Jet": "Jet",
    "Emperor": "Emperor",
    "Lynx": "Lynx",
    "Monkey_King": "Monkey King",
    "Viper": "Cobra",
}


letters = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
cols = letters.copy()
for i in range(len(letters)):
    for j in range(len(letters)):
        cols.append(letters[i] + letters[j])
        
skip = ["11.0", "12.0", "13.0", "1.0"]

In [3]:
import xlrd
import xlsxwriter
import pandas as pd
import openpyxl
from openpyxl import workbook
from openpyxl import load_workbook

pd.set_option("display.max_rows", None, "display.max_columns", None)

In [4]:
wb = load_workbook("BalanceHeroes/BalanceHeroesOld.xlsx")
    
analize = []

In [5]:
#def WriteNames(sheet, prows, pcols):
#    for name in names:
#        wb[name].insert_rows(prows + 1 + p[names.index(name)] - 3, len(names) + 5)
#    for name in names:
#        for i in range(len(names)):
#            wb[name].cell(row = prows + i + 1 + p[names.index(name)], column = pcols + 1).value = names[i]

def Analize(lvl, hero, heroEnemy, wr, oldwr):
    if wr < 0.4:
        analize.append("{0} {1} {2} {3:.0f}%".format(lvl, hero.replace(" ", ""), heroEnemy.replace(" ", ""), wr*100))
        print("{0} {1} {2} {3:.0f}%".format(lvl, hero.replace(" ", ""), heroEnemy.replace(" ", ""), wr*100))

def CopyWinrates():
    for name in names:
        sheet = wb[name]
        for i in range(len(names)):
            oldName = sheet.cell(row = i + 1 + p[names.index(name)], column = 1).value
            sheet.cell(row = names.index(oldName) + 1 + p2[names.index(name)], column = 1).value = oldName
            for j in range(9):
                oldWinrate = sheet.cell(row = i + 1 + p[names.index(name)], column = j*2 + 2).value
                oldDif = sheet.cell(row = i + 1 + p[names.index(name)], column = j*2 + 3).value
                sheet.cell(row = names.index(oldName) + 1 + p2[names.index(name)], column = j*2 + 2).value = oldWinrate
                sheet.cell(row = names.index(oldName) + 1 + p2[names.index(name)], column = j*2 + 3).value = oldDif
    
def OperateHero(sheet, heroName, startRow, prows, pcols):
    strong = 0
    weak = 0
    for row in reversed(range(1, startRow + 1)):
        if sheet.cell(row, 1).value != heroName or row == 1:
            break             
        winRate = sheet.cell(row, 4).value
        oldWinRate = wb[heroName].cell(row = prows + names.index(sheet.cell(row, 2).value) + 1 + p[names.index(heroName)], column = pcols + 1).value
        if oldWinRate == None:
            oldWinRate = 0
        if sheet.cell(row, 3).value >= 30:
            wb[heroName].cell(row = prows + names.index(sheet.cell(row, 2).value) + 1 + p[names.index(heroName)], column = pcols + 1).value = winRate
            wb[heroName].cell(row = prows + names.index(sheet.cell(row, 2).value) + 1 + p[names.index(heroName)], column = pcols + 2).value = winRate - oldWinRate 
           # Analize(sheet.cell(row, 0).value, heroName, sheet.cell(row, 2).value, winRate, 1)
        else:
            wb[heroName].cell(row = prows + names.index(sheet.cell(row, 2).value) + 1 + p[names.index(heroName)], column = pcols + 1).value = ""
            wb[heroName].cell(row = prows + names.index(sheet.cell(row, 2).value) + 1 + p[names.index(heroName)], column = pcols + 2).value = "" 
    wb[heroName].cell(row = prows + names.index(heroName) + 1 + p[names.index(heroName)], column = pcols + 1).value = ""

def OperateCategory(sheet, catName, startRow, prows, pcols):
    i = 0
    hero = ""
    for row in reversed(range(1, startRow)):
        if sheet.cell(row, 0).value != catName:
            break        
        if hero != sheet.cell(row, 1).value:
            hero = sheet.cell(row, 1).value
            OperateHero(sheet, hero, row, prows, pcols)
            i = i + 1
        
def DoBalanceTable(sheet, padding):
#    CopyWinrates()
    categories = []
    i = 0
    catName = ""
    for row in reversed(range(1, sheet.nrows)):
        if catName != sheet.cell(row, 0).value:
            categories.append(row)
            catName = sheet.cell(row, 0).value
            if not str(catName) in skip:
                OperateCategory(sheet, catName, row, padding, 1 + 2*i)
                i = i + 1
    return categories

def FindPaddings(book):
    p = []
    for sheet in wb.worksheets:
        i = 0
        for cell in sheet['A']:
            if cell.value == "#WINRATES":
                p.append(i + 4)
                break
            i = i + 1
    return p

#def FindPaddings2(book):
#    p2 = []
#    for sheet in wb.worksheets:
#        i = 0
#        for cell in sheet['A']:
#            if cell.value == "#WINRATES_2":
#                p2.append(i + 4)
#                break
#            i = i + 1
#    return p2

def FindPaddingsGeneral(book, name):
    p = []
    for sheet in wb.worksheets:
        i = 0
        for cell in sheet['A']:
            if cell.value == name:
                p.append(i + 4)
                break
            i = i + 1
    return p

In [6]:
p = FindPaddings(wb)
#p2 = FindPaddings2(wb)

In [7]:
# Винрейты
if do_balance:
    wbRD = xlrd.open_workbook("BalanceSource/Balance1_3_10_20.xlsx")
    sheets = wbRD.sheets()
    DoBalanceTable(sheets[3], 0)

In [8]:
#for name in names:
#    a = [str(x) for x in analize if x.split(" ")[1] == name.replace(" ", "")]
#    for i in range(len(a)):
#        a[i].split(" ")
#        print("{0} {1} {2} {3}".format(a[i].split(" ")[0], a[i].split(" ")[1], a[i].split(" ")[2], a[i].split(" ")[3]))

In [9]:
import pandas as pd
import os
import sys
import xlwt
import matplotlib.pyplot as plt

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from os import listdir
from os.path import isfile, join

In [10]:
class EhShasBiPoprogat:
    
    def __init__(self):
        text_file = open("kick_names.txt", "r", encoding='utf-8')
        self.names = text_file.read().split("\n")
        text_file = open("SameAttacks.txt", "r", encoding='utf-8')
        
        data = text_file.read()
        data = StringIO(data)

        self.df_same = pd.read_csv(data, sep=" ")
        self.df_same = self.df_same.apply(lambda x: x.str.lower())
        
        
    def is_shit(self, s):
        arr = s.split('_')
        arr = [x for x in arr if x.isdigit()]
        if len(arr)==0:
            return True
        if s.startswith("Shadow_Agile_Helmet_3"):
            return True
        return arr[-1]=='1'
    
    def is_move(self, s):
        return not s in self.names
    
    def is_not_move(self, s):
        return s in self.names
    
    def drop_shit(self, df):
        df = df[df["source"].map(self.is_shit)]
        df = df[~df["source"].str.startswith("lying_kick")]
        df = df[~df["source"].str.startswith("throw_forward_active")]
        df = df[~df["source"].str.startswith("throw_simple")]
        df = df.append(pd.DataFrame(data = {"source" : "legs", "amount" : df[df["source"].map(self.is_not_move)]["amount"].sum()} , index=[0]))
        df = df[~df["source"].apply(lambda x: any([x.startswith(name) for name in self.names]))]
        df = df[~df["source"].str.endswith("long")]
        df = df[~df["source"].str.startswith("dual_swords_heavy_slash")]
        df = df[~df["source"].apply(lambda x: any([x.startswith(name) for name in self.df_same["AddName"]]))]
        df = df[df.amount != 0]
        return df
    
    def operate_file(self, f, padding, padding_row, worksheet, byHero):
        if byHero:
            file = "DMG/actionsDmg/byHero/above3000" + "/" + f
        else:
            file = "DMG/actionsDmg/byMatchup/above3000" + "/" + f          
            
        fp = open(file, encoding='utf-8') 
        
        data = fp.read()
        
        lines = data.split("\n")
        data = [line for line in lines if "animation" not in line]
        data = '\n'.join([str(d) for d in data])
        
        data = StringIO(data)
        
        dftmp = pd.read_csv(data, sep=" ")
        
        #print(dftmp['source'])
        #print(dftmp['amount'])
        
        dfb = dftmp.loc["[B]"].reset_index(inplace=True, drop=True)
        dfbs = dftmp.loc["[BS]"].reset_index(inplace=True, drop=True)
        dfs = dftmp.loc["[S]"].reset_index(inplace=True, drop=True)
        df = dftmp.loc[dftmp.index.difference(["[B]", "[BS]", "[S]"])]
        df['amount'] = df['source']
        df['source'] = df.index
        df.reset_index(inplace=True, drop=True)
        df = df[~df["amount"].str.startswith("animation]")]
        df = df.astype({"amount": float})

        dfbuf = df
        for index, row in df.iterrows():
            add = 0
            tmp = self.df_same[self.df_same['AddName'].str.fullmatch(row['source'])]["MainName"]
            if not tmp.empty:
                add = dfbuf[dfbuf["source"].str.fullmatch(row['source'])].amount.values[0]
                dfbuf.loc[df.source == tmp.astype("string").values[0], 'amount'] += add
            if not row['source'] in self.df_same.values:
                dfbuf.at[index, 'amount'] = df.groupby(df["source"].str.startswith(row['source'][:-1]))["amount"].sum()[True]
        
        lying_kick_sum = df.groupby(df["source"].str.startswith("lying_kick"))["amount"].sum()[True]
        throws = df[(df.source == "throw_forward_active") | (df.source == "throw_simple")]["amount"].sum()
        df = self.drop_shit(dfbuf)
        df = df.append(pd.DataFrame([['lying_kick',lying_kick_sum], ['throws', throws]],columns=['source','amount'])).reset_index(drop=True)
        df.sort_values(by=['amount'], inplace=True, ascending=False)
        df.reset_index(inplace=True, drop=True)
        df
        plt.barh(df["source"], df["amount"])
        plt.title(label=f)
        plt.savefig("DMG/trash/" + f[:-4] + '.png', bbox_inches="tight")
        plt.clf()        
        img = openpyxl.drawing.image.Image('DMG/trash/'+ f[:-4] + '.png')
        img.anchor = cols[padding] + str(padding_row)
        worksheet.add_image(img)
        

In [11]:
def clear_space(sheet, padding, length):
    for i in range(length):
        row = sheet[padding + i]
        for cell in row:
            cell.value = None
            
def clear_category_general(book, category):
    pp = FindPaddingsGeneral(book, category)
    i = 0
    for sheet in book.worksheets:
        clear_space(sheet, pp[i], )
        i = i + 1
        
def delete_images_general(book):
    for sheet in book.worksheets:
        sheet._images = []
        #for img in sheet._images:
            #del img

In [12]:
# Стата по дамагу
if do_dmg:
    onlyfilesMain = [f for f in listdir("DMG/actionsDmg/byHero/above3000") if isfile(join("DMG/actionsDmg/byHero/above3000", f))]
    onlyfilesOther = [f for f in listdir("DMG/actionsDmg/byMatchup/above3000") if isfile(join("DMG/actionsDmg/byMatchup/above3000", f))]
  
    dmg = EhShasBiPoprogat()
  
    writer = pd.ExcelWriter('DMG/Dmg_plots_New.xlsx', engine = 'xlsxwriter')
    dfempty = pd.DataFrame([['','']],columns=['',''])       # ЭТО НЕ НУЖНО?
  
    padding = 2
  
    p3 = FindPaddingsGeneral(wb, "#DAMAGE_PLOTS")
  
    delete_images_general(wb)
    
    for f,i in zip(onlyfilesMain, range(len(onlyfilesMain))):
        dfempty.to_excel(writer, sheet_name = f)
        print(f)
        worksheet = wb[names_dmg[f[:-4]]]
        dmg.operate_file(f, padding, p3[names.index(names_dmg[f[:-4]])], worksheet, True)
        j = 0
        for ff,ii in zip(onlyfilesOther, range(len(onlyfilesOther))):
            if f == "Monk.txt" and ff.startswith("Monke"):
                continue
            if ff.startswith(f[:-4]):
                j = j + 1
                dmg.operate_file(ff, padding + 1 + j*6, p3[names.index(names_dmg[f[:-4]])], worksheet, False)
         #break
          
    writer.save()
    writer.close()

Bulwark.txt
Emperor.txt
Feldsher.txt
Fireguard.txt
Helga.txt
HongJoo.txt
Ironclad.txt
Jet.txt
Kibo.txt
Ling.txt
Liquidator.txt
Lynx.txt
Marcus.txt
Midnight.txt
Monk.txt
Monkey_King.txt
Sarge.txt
Viper.txt
Yukka.txt


<Figure size 432x288 with 0 Axes>

In [13]:
import pandas as pd
import os
import xlwt
import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from os import listdir
from os.path import isfile, join

In [14]:
class somebullshit:
    
    def __init__(self): 
        text_file = open("kick_names.txt", "r", encoding='utf-8')
        self.names = text_file.read().split("\n")
    
    def is_shit(self, s):
        arr = s.split('_')
        arr = [x for x in arr if x.isdigit()]
        if len(arr)==0:
            return True
        return arr[-1]=='1'
    
    def is_move(self, s):
        return not s in self.names
    
    def is_not_move(self, s):
        return s in self.names
    
    def vikinut_lishnyak(self, df):
        df = df[df["action"].map(self.is_shit)]
        df = df[~df["action"].str.contains("shadow")]
        df = df[~df["action"].str.contains("fly")]
        df = df[~df["action"].str.contains("rage")]
        df = df[~df["action"].str.contains("active")]
        df = df[~df["action"].str.contains("spit")]
        df = df[~df["action"].str.contains("shoot")]
        df = df[~df["action"].str.contains("pistol")]
        df = df[~df["action"].str.contains("throw")]
        df = df[~df["action"].str.contains("ranged")]
        lying_kick_sum = df.groupby(df["action"].str.startswith("lying_kick"))["count"].sum()[True]
        df = df[~df['action'].str.startswith('lying_kick')]
        df = df.append(pd.DataFrame([['lying_kick',lying_kick_sum]],columns=['action','count'])).reset_index(drop=True)
        return df
    
    def dolya(self, df, count_sum):
        arr = list(round(df[:-1]["count"] / df[:-1]["count"].sum(), 3).values)
        arr.append(round(df.iloc[-1]["count"]/count_sum, 3))
        df["dolya"] = arr
        asd = df[:-1].sort_values(by="dolya", ascending=False)
        asd = asd.append(df.iloc[-1]).reset_index(drop=True)
        return asd
            
    def toExcel(self, sh, df, row_padding, col_padding, source):
        sh.cell(row = row_padding + 1, column = col_padding).value = source
        
        for n, (v_desc, v) in enumerate(zip(df["action"][:-2], df["dolya"][:-2])):
            sh.cell(row = row_padding + 2 + n, column = col_padding).value = v_desc
            sh.cell(row = row_padding + 2 + n, column = col_padding + 1).value = v
        
        for n, (v_desc, v) in enumerate(zip(df["action"][-2:], df["dolya"][-2:])):
            sh.cell(row = row_padding + 23 + n, column = col_padding).value = v_desc
            sh.cell(row = row_padding + 23 + n, column = col_padding + 1).value = v            
        
        for n, (v_desc, v) in enumerate(zip(self.add["action"], self.add["dolya"])):
            sh.cell(row = row_padding + 30 + n, column = col_padding).value = v_desc
            sh.cell(row = row_padding + 30 + n, column = col_padding + 1).value = v

    def countlegs(self, df):
        df = df.append(pd.DataFrame(data = {'action' : "legs", 'count' : df[df['action'].map(self.is_not_move)]["count"].sum(), 'dolya' : df[df['action'].map(self.is_not_move)]["dolya"].sum()} , index=[0]))
        return df
    
    def additionalList(self, df):
        self.add = df[df["action"].map(self.is_move)][:-2]
        return self.add

In [15]:
def vneclassnaya_rabota(path, pathOther):
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    onlyfilesOther = [f for f in listdir(pathOther) if isfile(join(pathOther, f))]    
    
    for f,i in zip(onlyfiles, range(len(onlyfiles))):
        print(f)
        padding = 2
        worksheet = wb[names_dmg[f[:-4]]]
        
        file = path + "/" + f
        fp = open(file, encoding='utf-8')        
        data = fp.read()
        data = StringIO(data)        
        df = pd.read_csv(data, sep=" ")
        count_sum = df["count"].sum()        
        sb = somebullshit()        
        df = sb.vikinut_lishnyak(df)
        df = sb.dolya(df, count_sum)
        df = sb.countlegs(df)
        df.reset_index(drop=True)
        sb.additionalList(df)        
        
        sb.toExcel(worksheet, df, p4[names.index(names_dmg[f[:-4]])], padding, f)
        
        j = 0        
        for ff,ii in zip(onlyfilesOther, range(len(onlyfilesOther))):
            if f == "Monk.txt" and ff.startswith("Monke"):
                continue
            if ff.startswith(f[:-4]):
                j = j + 1
                
                file = pathOther + "/" + ff
                fp = open(file, encoding='utf-8')        
                data = fp.read()
                data = StringIO(data)        
                df = pd.read_csv(data, sep=" ")
                count_sum = df["count"].sum()        
                sb = somebullshit()        
                df = sb.vikinut_lishnyak(df)
                df = sb.dolya(df, count_sum)
                df = sb.countlegs(df)
                df.reset_index(drop=True)
                sb.additionalList(df)   
                
                sb.toExcel(worksheet, df, p4[names.index(names_dmg[f[:-4]])], padding + j*4, ff)
                

In [16]:
# Популярность ударов
if do_populyarnost:
    p4 = FindPaddingsGeneral(wb, "#POPULYARNOST")
    print(p4)

    vneclassnaya_rabota("Populyarnost/byHero/above3000/", "Populyarnost/byMatchup/above3000/")

[63, 63, 63, 63, 63, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 73, 68, 63]
Bulwark.txt
Emperor.txt
Feldsher.txt
Fireguard.txt
Helga.txt
HongJoo.txt
Ironclad.txt
Jet.txt
Kibo.txt
Ling.txt
Liquidator.txt
Lynx.txt
Marcus.txt
Midnight.txt
Monk.txt
Monkey_King.txt
Sarge.txt
Viper.txt
Yukka.txt


In [17]:
wb.save("BalanceHeroes/BalanceHeroesNew.xlsx") 